In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# !ls /content/drive/My\ Drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:

train_dir = '/content/drive/MyDrive/dataset_train'
test_dir = '/content/drive/MyDrive/dataset_test'
test_dir2 = '/content/drive/MyDrive/datasetMe'
grad_dir = '/content/drive/MyDrive/gradCam'
augment_dir = '/content/drive/MyDrive/augment_dir'
val_dir = '/content/drive/MyDrive/val_dir'
cam_dir = '/content/drive/MyDrive/CAM_test'

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil


img_size = 224
batch_size = 32
augmentation_save_dir = augment_dir
validation_save_dir = val_dir

os.makedirs(augmentation_save_dir, exist_ok=True)
os.makedirs(validation_save_dir, exist_ok=True)

datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

for category in os.listdir(train_dir):
    category_path = os.path.join(train_dir, category)
    save_augmentation_category_path = os.path.join(augmentation_save_dir, category)
    save_validation_category_path = os.path.join(validation_save_dir, category)
    os.makedirs(save_augmentation_category_path, exist_ok=True)
    os.makedirs(save_validation_category_path, exist_ok=True)

    if os.path.isdir(category_path):
        images = [img for img in os.listdir(category_path) if img.lower().endswith(('png', 'jpg', 'jpeg'))]
        num_validation = len(images) // 2

        # 分配到驗證集
        for img_name in images[:num_validation]:
            shutil.copy(os.path.join(category_path, img_name), os.path.join(save_validation_category_path, img_name))

        # 剩下的一半進行數據增強，放大三倍
        for img_name in images[num_validation:]:
            img_path = os.path.join(category_path, img_name)
            img = tf.keras.utils.load_img(img_path, target_size=(img_size, img_size))
            img_array = tf.keras.utils.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)

            # 保存原始圖片
            shutil.copy(img_path, os.path.join(save_augmentation_category_path, img_name))


            i = 0
            for batch in datagen.flow(img_array, batch_size=1, save_to_dir=save_augmentation_category_path,
                                      save_prefix='aug', save_format='jpeg'):
                i += 1
                if i >= 5:
                    break

print("augmentation and split done")



In [ ]:

train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    directory=augmentation_save_dir,
    target_size=(img_size, img_size),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    directory=validation_save_dir,
    target_size=(img_size, img_size),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)



In [ ]:
images, labels = next(train_generator)


num_images_to_display = 25

plt.figure(figsize=(8,8))
for i in range(num_images_to_display):
    plt.subplot(5,5, i + 1)
    plt.imshow(images[i].squeeze(), cmap='gray')
    plt.title(f"Label: {np.argmax(labels[i])}")
    plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.6)(x)
x = Dense(32, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.6)(x)
predictions = Dense(7, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)
model.summary()


In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import Callback

cam_dir = '/content/drive/MyDrive/CAM_test'

def grad_cam(model, image, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / (np.max(heatmap) + 1e-8)  # 確保不除以 0
    return heatmap


# 生成 Grad-CAM 疊加圖像
def generate_grad_cam_image(original_image, heatmap, alpha=0.4):
    heatmap = np.uint8(255 * heatmap)
    jet = plt.cm.get_cmap("jet")
    jet_heatmap = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_heatmap[heatmap]

    jet_heatmap = tf.image.resize(jet_heatmap, (original_image.shape[0], original_image.shape[1]))
    jet_heatmap = np.uint8(255 * jet_heatmap.numpy())


    superimposed_img = jet_heatmap * alpha + original_image / 255.0
    superimposed_img = np.clip(superimposed_img, 0, 1)
    return superimposed_img

class GradCamCallback(Callback):
    def __init__(self, images, labels, last_conv_layer_name, save_dir="gradcam_output"):
        super(GradCamCallback, self).__init__()
        self.images = images  
        self.labels = labels  # 圖片對應的真實標籤
        self.last_conv_layer_name = last_conv_layer_name
        self.save_dir = save_dir
        os.makedirs(self.save_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        for i, image in enumerate(self.images):
            heatmap = grad_cam(self.model, np.expand_dims(image, axis=0), self.last_conv_layer_name)  # 使用 self.model
            grad_cam_image = generate_grad_cam_image(image, heatmap)


            plt.figure(figsize=(5, 5))
            plt.imshow(grad_cam_image)
            plt.title(f"True Label: {self.labels[i]}")
            plt.axis('off')
            plt.savefig(os.path.join(self.save_dir, f"epoch_{epoch + 1}_img_{self.labels[i]}.png"))
            plt.close()
        print(f"Grad-CAM images for epoch {epoch + 1} saved to {self.save_dir}")


sample_dir = cam_dir
image_paths = [os.path.join(sample_dir, fname) for fname in os.listdir(sample_dir)]
sample_images = []
sample_labels = []

for img_path in image_paths:
    label = os.path.splitext(os.path.basename(img_path))[0]
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    sample_images.append(img_array)
    sample_labels.append(label)

sample_images = np.array(sample_images)


grad_cam_callback = GradCamCallback(
    # model=model,
    images=sample_images,
    labels=sample_labels,
    last_conv_layer_name="Conv_1",
    save_dir= grad_dir
)



In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=8,
    restore_best_weights=True
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

history_initial = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    batch_size=128,
    callbacks=[grad_cam_callback,early_stopping, reduce_lr]
)


for layer in base_model.layers[-20:]:
    layer.trainable = True


model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    batch_size=32,
    callbacks=[grad_cam_callback,early_stopping, reduce_lr]
)

model.save("FER_900_data_mobilenetv2.h5")


In [ ]:
# 7 emotions x 4 people x 1 picture	  = 28
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
import matplotlib.pyplot as plt


initial_epochs = len(history_initial.history['accuracy'])
total_epochs = initial_epochs + len(history_finetune.history['accuracy'])


plt.figure(figsize=(12, 6))
plt.plot(range(1, initial_epochs + 1), history_initial.history['accuracy'], label='Initial Training Accuracy')
plt.plot(range(1, initial_epochs + 1), history_initial.history['val_accuracy'], label='Initial Validation Accuracy')
plt.plot(range(initial_epochs + 1, total_epochs + 1), history_finetune.history['accuracy'], label='Fine-tuning Training Accuracy', linestyle='--')
plt.plot(range(initial_epochs + 1, total_epochs + 1), history_finetune.history['val_accuracy'], label='Fine-tuning Validation Accuracy', linestyle='--')
plt.legend()
plt.title("Model Accuracy Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.grid()
plt.show()


plt.figure(figsize=(12, 6))
plt.plot(range(1, initial_epochs + 1), history_initial.history['loss'], label='Initial Training Loss')
plt.plot(range(1, initial_epochs + 1), history_initial.history['val_loss'], label='Initial Validation Loss')
plt.plot(range(initial_epochs + 1, total_epochs + 1), history_finetune.history['loss'], label='Fine-tuning Training Loss', linestyle='--')
plt.plot(range(initial_epochs + 1, total_epochs + 1), history_finetune.history['val_loss'], label='Fine-tuning Validation Loss', linestyle='--')
plt.legend()
plt.title("Model Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.grid()
plt.show()
